#### INI

In [1]:
import pandas as pd
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

In [13]:
# param d'app
SpotifyClientId="XXX"
SpotifyClientSecret="XXX"

# param compte
scope = 'playlist-modify-public playlist-modify-private playlist-read-private'
username = 'XXX'
redirect_uri = "https://localhost:8888/callback"

client_credentials_manager = SpotifyClientCredentials(SpotifyClientId, SpotifyClientSecret)

auth_manager = SpotifyOAuth(scope=scope,
                     username=username,
                     client_id=SpotifyClientId,
                     client_secret=SpotifyClientSecret,
                     redirect_uri=redirect_uri)

spotifyObject_search = Spotify(client_credentials_manager = client_credentials_manager)

spotifyObject = Spotify(auth_manager = auth_manager)

#### TOUTES LES FONCTIONS

In [43]:
 #1.
# CRÉER UNE NOUVELLE PLAYLIST
def create_playlist(playlist_name, playlist_description):
    
    playlist = spotifyObject.user_playlist_create(user=username, name=playlist_name, public=True, description=playlist_description)
    return playlist['id']

 #2.
# AFFICHE TOUTES LES PLAYLISTS D'UN UTILISATEUR 
def get_playlists(limit = 10):

    access_playlist=spotifyObject.user_playlists(user=username, limit=limit)
    playlist=access_playlist['items'][0]['id']
    display_playlists=access_playlist['items']
    for item in display_playlists:
        print()
        print("name: ", item['name'], " - ", "id: ", item['id'])
        print("description: ", item['description'])    
 
 #3.
# AFFICHE TOUTES LES TRACKS D'UNE PLAYLIST SPÉCIFIQUE
def display_tracks(playlist_id):
    
    display_tracks=spotifyObject.user_playlist_tracks(user=username, playlist_id=playlist_id)
    access_playlist=spotifyObject.playlist(playlist_id=playlist_id)
    playlist_to_update=access_playlist['name']
    print("\n", len(access_playlist['tracks']['items']) , "TRACKS TROUVÉES DANS LA PLAYLIST", playlist_to_update, ": ", "\n")
    for track in display_tracks["items"]:
        track_name = track["track"]["name"]
        track_artist = track["track"]["artists"][0]["name"]
        track_uri =  track["track"]["uri"]
        print(" - ", track_name, "par", track_artist)
        print("track uri:", track_uri, "\n")
    return track_uri


 #4.
# UNIQUEMENT AJOUTER DES TRACKS DANS UNE PLAYLIST
def add_new_tracks(playlist_id, track_uris):
    spotifyObject.user_playlist_add_tracks(user=username, playlist_id=playlist_id, tracks=track_uris)
   

 #5.   
# UNIQUEMENT SUPPRIMER DES TRACKS DANS UNE PLAYLIST
def remove_tracks(playlist_id, track_uris):
    spotifyObject.user_playlist_remove_all_occurrences_of_tracks(user=username, playlist_id=playlist_id, tracks=track_uris)
    

 #6.
# METTRE À JOUR TOUTE LA PLAYLIST
#remplace toutes les tracks par des nouvelles
def update_all(playlist_id, track_uris):
    spotifyObject.user_playlist_replace_tracks(user=username, playlist_id=playlist_id, tracks=track_uris)
    
 #7.   
# METTRE À JOUR LE TITRE + DESC DE LA PLAYLIST
def update_playlist_details(playlist_id, playlist_name, playlist_description):
    
    spotifyObject.user_playlist_change_details(user=username, playlist_id=playlist_id, name=playlist_name, public=True, description=playlist_description)
    new_playlist_info=spotifyObject.user_playlist(user=username, playlist_id=playlist_id)
    print("\n", "Nouveau nom:", new_playlist_info['name'])
    print(" Nouvelle description:", new_playlist_info['description'])
    
   
 #8.
# RECHERCHER DES TRACKS
# retourne l'id/uri d'une track spécifique
# sinon affiche [WARNING]

def search_track(track_input):
    
    result = spotifyObject_search.search(track_input)
    
    items = result['tracks']['items']

    artists = [i["artists"][0] for i in items]
    artists_names = [a["name"] for a in artists]

    tracks_names = [n["name"] for n in items]

    tracks_artists_names = list(zip(tracks_names,artists_names))
    tracks_artists_names = [ t+" - "+a for t,a in tracks_artists_names]
    
    if(len(set(tracks_artists_names)) == 1):
        print("[SUCCESS] RESULTAT POUR", track_input)
        track_uris = items[0]['uri']
        return track_uris
        
    elif(len(set(tracks_artists_names)) == 0):
        print("[WARNING] PAS DE RESULTATS POUR", track_input)
        
    else:
        print("[WARNING] IL Y A PLUSIEURS RESULTATS POUR", track_input)
        for item in items:
            print("\t", item['name'], " - ", item['artists'][0]['name'])

#### CRÉER / METTRE À JOUR LA PLAYLIST

In [49]:
playlist_id = create_playlist("pauline", "tous les morceaux avec pauline dedans")

uris = []
for _, row in df.iterrows():
    value = row["name"] + " " + row["primary_artist_name"]
    
    uri = search_track(value)
    if(uri != None):
        uris.append(uri)
          
update_all(playlist_id, uris)

[WARNING] PAS DE RESULTATS POUR Hard Amat’ Svinkels
[SUCCESS] RESULTAT POUR Les Vestiges de ma Vingtaine Fayçal
[SUCCESS] RESULTAT POUR Balance Gal Lord Kossity
[WARNING] IL Y A PLUSIEURS RESULTATS POUR Bicarbonate Fally Ipupa
	 Bicarbonate  -  Fally Ipupa
	 Bicarbonate - Live  -  Fally Ipupa
	 Bicarbonate - Live  -  Fally Ipupa
[SUCCESS] RESULTAT POUR Chaque fois que j’y pense Fisto
[WARNING] PAS DE RESULTATS POUR Wojtek Vs Srin Po Rap Contenders
[SUCCESS] RESULTAT POUR Elyxir Vald
[WARNING] PAS DE RESULTATS POUR PÉQUISSS Les Anticipateurs
[WARNING] PAS DE RESULTATS POUR PÉQUISSS Les Anticipateurs
[WARNING] PAS DE RESULTATS POUR PÉQUISSS Les Anticipateurs
[WARNING] PAS DE RESULTATS POUR PÉQUISSS Les Anticipateurs
[WARNING] PAS DE RESULTATS POUR Dony S vs Wojtek Rap Contenders
[WARNING] PAS DE RESULTATS POUR Dony S vs Wojtek Rap Contenders
[WARNING] PAS DE RESULTATS POUR Blackapar vs Sango Rap Contenders
[WARNING] PAS DE RESULTATS POUR Lawid & Dinos Punchlinovic vs Lunik & Mic Orni Rap

[WARNING] PAS DE RESULTATS POUR VI - Pauline Totorhina
[SUCCESS] RESULTAT POUR Guiseppe Lyonzon
[SUCCESS] RESULTAT POUR Pauline Swenz
[SUCCESS] RESULTAT POUR Back To The Sippin Lyonzon
[WARNING] PAS DE RESULTATS POUR Moet RBstreet
[WARNING] PAS DE RESULTATS POUR Les Autres (Intro) T.I.S
[SUCCESS] RESULTAT POUR Xavi Lyonzon
[WARNING] PAS DE RESULTATS POUR K5 vs Louvar (Battle pour le titre) Rap Contenders
[SUCCESS] RESULTAT POUR Saut à la perche Raccoon
[WARNING] PAS DE RESULTATS POUR Nodcd Lezio
[WARNING] PAS DE RESULTATS POUR Glitch #1 Lezio
[SUCCESS] RESULTAT POUR Bruce Wayne Josman
[SUCCESS] RESULTAT POUR Vie rapide Swenz
[WARNING] PAS DE RESULTATS POUR Freestyle Savies #1 - Drill KaNoé (BEL)
[WARNING] PAS DE RESULTATS POUR SLT Noname (FRA)
[SUCCESS] RESULTAT POUR 13 balles 13 Organisé
[SUCCESS] RESULTAT POUR Vanille Axelence
[WARNING] PAS DE RESULTATS POUR Cheef vs. Sedj ROAR (Battle Rap)
[SUCCESS] RESULTAT POUR Lipgloss Loveni
[SUCCESS] RESULTAT POUR Nickel Le Croc
[SUCCESS] RESUL